<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/Modele000002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

##Python / Colab
from google.colab import files
from google.colab import drive
import os
from IPython.display import Image as imgIPython
from IPython.display import clear_output,display
import IPython
## Tensorflow keras
try:
  !pip install -q tf-nightly
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.python import debug as tf_debug
from tensorflow.python.client import device_lib

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,Conv2D,Convolution2D,Activation,Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D
from tensorflow.keras.layers import Dropout,Reshape,BatchNormalization
from tensorflow.keras.layers import concatenate,Concatenate,Subtract,Multiply,Average,Add
from tensorflow.keras.layers import UpSampling2D, Reshape,Flatten
from tensorflow.keras.layers import Lambda

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.models import Model
import tensorflow.keras.losses
## Math libraries
import numpy as np
import scipy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
##Images
from PIL import Image
import cv2
## Graph
from graphviz import render
from graphviz import Digraph,Graph
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
#Dataset
from __future__ import absolute_import, division, print_function, unicode_literals
import random
import pathlib
import shutil
import time
#Debugage
from IPython.display import clear_output
#Hyperparameters tuning
# !pip install -U keras-tuner #De https://github.com/keras-team/keras-tuner
from tensorboard.plugins.hparams import api as hp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE


#Dataset

In [0]:
#@title { display-mode: "form" }

dossier_TIPE = "/content/drive/My Drive/TIPE/"#@param {type:"string"}

Vérification des datasets

In [0]:
path = dossier_TIPE+"Galaxies_resized/"
dossiers = ["Train","Validation","Test"]
compte = []
for d in dossiers:
    compte.append(len(os.listdir(path+d)))
print([c/sum(compte) for c in compte])
print([c/10 for c in compte])

[0.6089108910891089, 0.297029702970297, 0.09405940594059406]
[24.6, 12.0, 3.8]


Création des classes de dataset Tensorflow

In [0]:
#D'après https://stackoverflow.com/questions/54590363/create-tensorflow-dataset-from-image-local-directory
class ArtificialDataset(tf.data.Dataset):
    def open_imgs(x):
        img = tf.io.read_file(x)
        img = tf.image.decode_jpeg(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, [256, 256])
        img = tf.clip_by_value(img,0,1)
        return img
        
    def _generator():
        for img_batch in ArtificialDataset.donnees:
            cleans = []
            for i,img in enumerate(img_batch):
                clean = ArtificialDataset.open_imgs(img)
                cleans.append(clean)
            cleans_t = tf.stack(cleans)
            yield (cleans_t)
    
    def __new__(cls,nom):
        ArtificialDataset.donnees = tf.data.Dataset.list_files(dossier_TIPE+"Galaxies_resized/"+nom+"/*.jpg").shuffle(len(os.listdir(dossier_TIPE+"Galaxies_resized/"+nom))).batch(10)
        #voir doc from_generator https://github.com/tensorflow/tensorflow/blob/f2b2563c6ce2001a117cd7adb36f303903e907ec/tensorflow/python/data/ops/dataset_ops.py#L669
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.dtypes.float32),
            output_shapes=(tf.TensorShape([None,256,256,3])), #https://www.tensorflow.org/api_docs/python/tf/TensorShape?version=nightly
            args=None
        )

In [0]:

#Bruit gaussien origine : https://stackoverflow.com/questions/59286171/gaussian-blur-image-in-dataset-pipeline-in-tensorflow
def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
    """Génère le noyau de la couche de convolution du flou gaussien"""
    x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
    g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
    g_norm2d = tf.pow(tf.reduce_sum(g), 2)
    g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
    g_kernel = tf.expand_dims(g_kernel, axis=-1)
    return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)

def apply_blur(img):
    """Applique le flou gaussien"""
    img = tf.reshape(img,[1,256,256,3])
    blur = _gaussian_kernel(3, 2, 3, img.dtype)
    img = tf.nn.depthwise_conv2d(img, blur, [1,1,1,1], 'SAME')
    return tf.reshape(img,[256,256,3])
def bruitage(img):
    #Bruitage
    img_noise = tf.image.adjust_saturation(img,.8)
    img_noise = apply_blur(img_noise)
    img_noise = tf.image.adjust_contrast(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.1,maxval=0.2)))
    ajout = 0.1
    img_noise = img_noise+tf.concat([tf.ones(shape=[256,256,1])*ajout,tf.zeros(shape=[256,256,2])],axis=-1)
    img_noise = tf.image.adjust_brightness(img_noise,-ajout)
    img_noise = tf.image.adjust_hue(img_noise,tf.reduce_sum(tf.random.uniform(shape=(1,),dtype=tf.float32,minval=0.06,maxval=0.09)))
    img_noise = tf.clip_by_value(img_noise,0,1)
    return img_noise
def traitement(img_batch):
    """Modification par image"""
    noised = tf.map_fn(bruitage,img_batch)
    img_noise_batch = tf.stack([tf.slice(noised,[0,0,0,0],[len(img_batch)//2,256,256,3]),tf.slice(img_batch,[len(img_batch)//2,0,0,0],[len(img_batch)-len(img_batch)//2,256,256,3])])
    return img_batch,img_noise_batch

# Modèle

## Couche de convolution et déconvolution liée

In [0]:
class Conv2DTranspose_tied(Conv2DTranspose):
    def __init__(self,
               filters,
               kernel_size,
               strides=(1, 1),
               padding='valid',
               output_padding=None,
               data_format=None,
               dilation_rate=(1, 1),
               activation=None,
               use_bias=True,
               kernel_initializer='glorot_uniform',
               bias_initializer='zeros',
               kernel_regularizer=None,
               bias_regularizer=None,
               activity_regularizer=None,
               kernel_constraint=None,
               bias_constraint=None,
               tied_to = None,
               **kwargs):
        self.tied_to = tied_to
        self.initial_weights = tied_to.initial_weights
        super(Conv2DTranspose_tied, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activations.get(activation),
            use_bias=use_bias,
            kernel_initializer=initializers.get(kernel_initializer),
            bias_initializer=initializers.get(bias_initializer),
            kernel_regularizer=regularizers.get(kernel_regularizer),
            bias_regularizer=regularizers.get(bias_regularizer),
            activity_regularizer=regularizers.get(activity_regularizer),
            kernel_constraint=constraints.get(kernel_constraint),
            bias_constraint=constraints.get(bias_constraint),
            **kwargs)
    def build(self,input_shape):
        super(Conv2DTranspose_tied, self).build(self,input_shape)
        self.initial_weights = tied_to.initial_weights
    def call(self, inputs):
        self.kernel = tf.transpose(self.tied_to.kernel, (1, 0, 2, 3))
        if self.bias:
            if self.dim_ordering == 'th':
                output += K.reshape(self.b, (1, self.nb_filter, 1, 1))
            elif self.dim_ordering == 'tf':
                output += K.reshape(self.b, (1, 1, 1, self.nb_filter))
            else:
                raise Exception('Invalid dim_ordering: ' + self.dim_ordering)
        output = self.activation(output)
        return output
        if self._recreate_conv_op(inputs):
            self._convolution_op = nn_ops.Convolution(
                inputs.get_shape(),
                filter_shape=self.kernel.shape,
                dilation_rate=self.dilation_rate,
                strides=self.strides,
                padding=self._padding_op,
                data_format=self._conv_op_data_format)

        # Apply causal padding to inputs for Conv1D.
        if self.padding == 'causal' and self.__class__.__name__ == 'Conv1D':
            ts = array_ops.pad(inputs, self._compute_causal_padding())

        outputs = self._convolution_op(inputs, self.kernel)

        if self.use_bias:
            if self.data_format == 'channels_first':
                if self.rank == 1:
                    # nn.bias_add does not accept a 1D input tensor.
                    bias = array_ops.reshape(self.bias, (1, self.filters, 1))
                    outputs += bias
                else:
                    outputs = nn.bias_add(outputs, self.bias, data_format='NCHW')
        else:
            outputs = nn.bias_add(outputs, self.bias, data_format='NHWC')

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

# Autoencoeur simple

On met en place de l'optimisation bayesienne pour former le reseau au mieux sachant que :

L'image fait $$256=2⁸$$ pixels 

un max pooling donnant $$output = \frac{output}{2}$$ avec un stride de 2 et un noyau de 2

une couche de déconvolution donne :


$$newSize = (size - 1)\cdot stride + k$$ sans padding











On peut donc avoir entre 0 et 8 réductions de taille

Donc 
- 0 --> 8 Pooling avec `strides=2, padding='VALID', kernel=2` 
- couches de convolution
    - en nombre libre de toute contrainte théoriquement mais `en pratique on cherchera à en minimiser le nombre
    - étant liée `obligatoirement à la couche suivante` et peut-être `à d'autres couche de même taille d'image`
    - `kernel` entre 1 et 3
    - nombre de `filtre` entre 1 et 500
- autant de deconvolution que de pooling
    - `kernel=2`
    - `stride=2`

L'idée est de faire un modèle pas trop compliqué pour commencer càd :
- sans batch normalisation
- en conséquence en limitant la profondeur en limitant le nombre de couche de convolution

In [0]:
#{ display-mode: "form" }
nb_max_conv_par_etage = 3#@param {type:'number'}
possibilites_filtres = [1,3,10,50,100,500]#@param {type:'string'}

In [0]:
def modele(hparams):
    input = Input(shape=(256,256,3),name='Entree_10x256x256x3')
    Lparam = []
    couche,Lparam = convolution(hparams,0,input,'desc',Lparam)
    nb_pool = hp.HParam('nb_pool',hp.IntInterval(min_value=0,max_value=9))
    Lparam.append(nb_pool)
    for index_pooling in range(hparams[nb_pool]):
        type_pool = hp.HParam('pool_type_%d'%index_pool,hp.Discrete(['avg','max']))
        Lparam.append(type_pool)
        if hparams[type_pool] == 'avg':
            couche = AveragePooling2D(pool_size=2,strides=2,padding='VALID',name='AveragePool')
        else:
            couche = MaxPooling2D(pool_size=2,strides=2,padding='VALID',name='MaxPool')
        couche,LparamSortie = convolution(hparams,index_pooling+1,couche,Lparam)
        Lparam = LparamSortie
    for index_deconv in range(hparams[nb_pool]):
        filters = hp.HParam('filtre_deconv_etg_%d_%s'%(index_deconv,'remont'),hp.Discrete(possibilites_filtres))
        Lparam.append(filters)
        couche = Conv2DTranspose(filters=hparams[filters],kernel_size=2,strides=2,name='TransposedConv_Deconv_f%d'%hparams[filters])
        couche,LparamSortie = convolution(hparams,index_deconv,couche,'remont',Lparam)
        Lparam = LparamSortie
    couche = Conv2D(filters=3,kernel_size=2,padding='SAME',name='Convolution_k%d_f%d_activ_%s'%(3,2,'linear'))
    model = Model(inputs=input,outputs=couche,name='Debruiteur')
    lr = hp.HParam('learning_rate',hp.Discrete( [1e-2, 1e-3, 1e-4]))
    beta1 = hp.HParam('beta_1',hp.Float(8e-1,1.2))
    beta2 = hp.HParam('beta_2',hp.Float(8e-1,1.2))
    Lparam.append(lr)
    Lparam.append(beta1)
    Lparam.append(beta2)
    model.compile(optimiszer=tf.keras.optimizer.Adam(hparams[lr],
                                                     beta_1=hparams[beta1],
                                                     beta_2=hparams[beta2],
                  loss='MSE',
                  metrics=['accuracy'])
    with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
        hp.hparams_config(
            hparams=Lparam,
            metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
        )

    return model
def convolution(hparams,etage,input,pool_txt,Lparam):
    couche = input
    L_conv_param = []
    nb_conv = hp.HParam('nb_conv_etg_%d_%s'%(etage,pool_txt),hp.IntInterval(min_value=0,max_value=nb_max_conv_par_etage+1))
    Lparam.append(nb_conv)
    for index_conv in range(hparams[nb_conv]):
        filters = hp.HParam('filtre_conv_index_%d_etg_%d_%s'%(index_conv,etage,pool_txt),hp.Discrete(possibilites_filtres))
        kernel = hp.HParam('kernel_conv_index_%d_etg_%d_%s'%(index_conv,etage,pool_txt),hp.Discrete([2,3]))
        activ = hp.HParam("activation_conv_index_%d_etg_%d_%s"%(index_conv,etage,pool_txt),hp.Discrete([None,'relu','elu','selu','tanh']))
        Lparam.append(filters)
        Lparam.append(kernel)
        Lparam.append(activ)
        couche = Conv2D(filters=hparams[filters],
                             kernel_size=hparams[kernel],
                             padding='SAME',
                             activation=hparams[activ],
                             name='Convolution_k%d_f%d_activ_%s'%(hparams[filters],hparams[kernel],hparams[activ] if type(hparams[activ])==str else 'linear'))(couche)
    return couche,Lparam